<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Natural_language_to_sql_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU llama_index sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing

In [ ]:
!pip install openai

## Instantiate the Openai keys

In [ ]:
from google.colab import userdata
import openai
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
openai_api_key = userdata.get('OPENAI_API_KEY')


In [ ]:
import llama_index

## Load the CSV file to pandas dataframe

In [ ]:
import pandas as pd
df = pd.read_csv("/content/BA_AirlineReviews.csv")
df.head()

,Unnamed: 0,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
0,0,1.0,"""Service level far worse then Ryanair""",L Keele,19th November 2023,True,4 Hours before takeoff we received a Mail stat...,Couple Leisure,Economy Class,London to Stuttgart,November 2023,1.0,1.0,1.0,1.0,no,NaN,NaN,NaN,NaN
1,1,3.0,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2.0,3.0,1.0,2.0,no,A320,1.0,2.0,2.0
2,2,8.0,"""Flight was smooth and quick""",M A Collie,16th November 2023,False,"Boarded on time, but it took ages to get to th...",Couple Leisure,Business Class,London Heathrow to Dublin,November 2023,3.0,3.0,4.0,3.0,yes,A320,4.0,NaN,NaN
3,3,1.0,"""Absolutely hopeless airline""",Nigel Dean,16th November 2023,True,"5 days before the flight, we were advised by B...",Couple Leisure,Economy Class,London to Dublin,December 2022,3.0,3.0,1.0,1.0,no,NaN,NaN,NaN,NaN
4,4,1.0,"""Customer Service is non existent""",Gaylynne Simpson,14th November 2023,False,"We traveled to Lisbon for our dream vacation, ...",Couple Leisure,Economy Class,London to Lisbon,November 2023,1.0,1.0,1.0,1.0,no,NaN,1.0,1.0,1.0


## Drop the unwanted columns

In [ ]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
0,1.0,"""Service level far worse then Ryanair""",L Keele,19th November 2023,True,4 Hours before takeoff we received a Mail stat...,Couple Leisure,Economy Class,London to Stuttgart,November 2023,1.0,1.0,1.0,1.0,no,NaN,NaN,NaN,NaN
1,3.0,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2.0,3.0,1.0,2.0,no,A320,1.0,2.0,2.0
2,8.0,"""Flight was smooth and quick""",M A Collie,16th November 2023,False,"Boarded on time, but it took ages to get to th...",Couple Leisure,Business Class,London Heathrow to Dublin,November 2023,3.0,3.0,4.0,3.0,yes,A320,4.0,NaN,NaN
3,1.0,"""Absolutely hopeless airline""",Nigel Dean,16th November 2023,True,"5 days before the flight, we were advised by B...",Couple Leisure,Economy Class,London to Dublin,December 2022,3.0,3.0,1.0,1.0,no,NaN,NaN,NaN,NaN
4,1.0,"""Customer Service is non existent""",Gaylynne Simpson,14th November 2023,False,"We traveled to Lisbon for our dream vacation, ...",Couple Leisure,Economy Class,London to Lisbon,November 2023,1.0,1.0,1.0,1.0,no,NaN,1.0,1.0,1.0


## Check the % of null values with each column attribute

In [ ]:
df.isnull().sum()[df.isnull().sum() > 0] / len(df) * 100.0

OverallRating             0.135099
TypeOfTraveller          20.832208
SeatType                  0.054039
Route                    20.940286
DateFlown                21.021346
SeatComfort               3.134288
CabinStaffService         3.431505
GroundService            22.858687
ValueForMoney             0.027020
Aircraft                 48.068090
Food&Beverages           10.429614
InflightEntertainment    31.072683
Wifi&Connectivity        83.544988
dtype: float64

## drop the null values

In [ ]:
temp =df.dropna()
temp.isnull().sum()[temp.isnull().sum() > 0] / len(temp) * 100.

Series([], dtype: float64)

In [ ]:
df.shape, temp.shape

((3701, 19), (327, 19))

In [ ]:
temp.head()

,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
1,3.0,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2.0,3.0,1.0,2.0,no,A320,1.0,2.0,2.0
8,2.0,"""Angry, disappointed, and unsatisfied""",Massimo Tricca,5th November 2023,False,"Angry, disappointed, and unsatisfied. My route...",Family Leisure,Economy Class,London Heatrow to Atlanta,November 2023,4.0,5.0,3.0,5.0,yes,Boeing 777,4.0,4.0,3.0
25,5.0,"""Club Europe is simply a joke""",M Dale,14th October 2023,True,I am a frequent flyer with BA and have been fo...,Business,Business Class,London to Istanbul,October 2023,3.0,4.0,3.0,2.0,no,A320,1.0,1.0,1.0
33,10.0,"""Excellent service levels""",Peter Costello,7th October 2023,True,"Excellent service levels, proactive crew and s...",Solo Leisure,First Class,London to New York JFK,October 2023,5.0,5.0,5.0,5.0,yes,Boeing 777,5.0,4.0,5.0
34,1.0,"""British Airways was absolutely shocking""",Kane Kelly,5th October 2023,False,Booked a very special holiday for me and my pa...,Couple Leisure,Business Class,Heathrow to Marseille,August 2023,1.0,1.0,1.0,1.0,no,BA366,1.0,1.0,1.0


## Create SQLAlchemy engine with SQLlite and convert dataframe into SQL tables

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
temp.to_sql('reviews', con=engine)

327

## Construct a SQLDatabase Index

In [ ]:
from llama_index import SQLDatabase
reviews_db = SQLDatabase(engine,include_tables=['reviews'])

## Instantiate the LLLM - Openai in our case

In [ ]:
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext
llm = OpenAI(model_name="gpt-4-0613",
             temperature=0,
             streaming=True)
embedding = OpenAIEmbedding()
#
servicecontext = ServiceContext.from_defaults(llm=llm,
                                embed_model=embedding,
                                chunk_size=500)

## Create the Natural Language to SQL table query engine for the added SQL table

In [85]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
sql_query_engine = NLSQLTableQueryEngine(sql_database=reviews_db,
                                         tables=['reviews'],
                                         service_context=servicecontext)

## Add a detailed description making sense for the NLSQLTableQueryEngine

In [88]:
description = ("Provides information about airlines reviews from reviews table. "
                "Use a detailed plain text question as input to the tool.")

## wrap all the details into QueryEngineTool

In [89]:
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.agent import OpenAIAgent
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)


In [65]:
agent

## Generate the response

In [66]:
response = agent.chat("What are the top 10 problems mentioned by our customers?")

Added user message to memory: What are the top 10 problems mentioned by our customers?
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT problem, COUNT(*) as count FROM reviews GROUP BY problem ORDER BY count DESC LIMIT 10"
}
Got output: I'm sorry, but there seems to be an error with the SQL statement. Please check the syntax and try again.

=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT problem, COUNT(*) as count FROM reviews GROUP BY problem ORDER BY count DESC LIMIT 10"
}
Got output: I'm sorry, but there seems to be an error with the SQL statement. Please check the syntax and try again.

=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT problem, COUNT(*) as count FROM reviews GROUP BY problem ORDER BY count DESC LIMIT 10"
}
Got output: I'm sorry, but there seems to be an error with the SQL statement. Please check the syntax and try again.

=== Calling Function ===
Calling func

In [69]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("What is the highest overall rating provided by travellers?")

Added user message to memory: What is the highest overall rating provided by travellers?
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT MAX(overall_rating) AS highest_rating FROM reviews"
}
Got output: The highest overall rating from the reviews is 10.0.



In [72]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("How many Business class travellers have given overall ratings greater than 3?")

Added user message to memory: How many Business class travellers have given overall ratings greater than 3?
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT COUNT(*) FROM reviews WHERE class = 'Business' AND overall_rating > 3"
}
Got output: There are no reviews in the database that have a seat type of "Business" and an overall rating greater than 3.



In [74]:
str(response)

'There are no Business class travelers who have given overall ratings greater than 3.'

In [77]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("What is the review mentioned for the Route from 'London Heatrow to Atlanta' ?")

Added user message to memory: What is the review mentioned for the Route from 'London Heatrow to Atlanta' ?
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT review FROM airline_reviews WHERE route = 'London Heathrow to Atlanta'"
}
Got output: There are no reviews available for the London Heathrow to Atlanta route.



In [78]:
str(response)

'There are no reviews available for the route from London Heathrow to Atlanta.'

In [80]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("What is the review mentioned by name 'Austin Jones'?")

Added user message to memory: What is the review mentioned by name 'Austin Jones'?
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT * FROM reviews WHERE name = 'Austin Jones'"
}
Got output: The review from Austin Jones states that they recently experienced a delay on a British Airways flight from Brussels to London. The delay was due to staff shortages and resulted in a 2-hour holding delay. Austin Jones mentions that British Airways has declined in recent years and has become more like a budget airline with worse service and timeliness compared to Ryanair or EasyJet. The review was categorized as "Business" and the flight was in "Economy Class". The review was posted in November 2023 and received ratings of 2.0 for overall experience, 3.0 for seat comfort, 1.0 for cabin staff service, and 2.0 for food and beverage service. The reviewer did not upgrade their seat and the aircraft used for the flight was an A320.



In [81]:
str(response)

'The review from Austin Jones states that they recently experienced a delay on a British Airways flight from Brussels to London. The delay was due to staff shortages and resulted in a 2-hour holding delay. Austin Jones mentions that British Airways has declined in recent years and has become more like a budget airline with worse service and timeliness compared to Ryanair or EasyJet. The review was categorized as "Business" and the flight was in "Economy Class". The review was posted in November 2023 and received ratings of 2.0 for overall experience, 3.0 for seat comfort, 1.0 for cabin staff service, and 2.0 for food and beverage service. The reviewer did not upgrade their seat and the aircraft used for the flight was an A320.'

In [90]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("Wich Route	 has the highest OverallRating ? ")

Added user message to memory: Wich Route	 has the highest OverallRating ? 
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT Route, MAX(OverallRating) AS HighestOverallRating FROM reviews"
}
Got output: The route with the highest overall rating is London to New York JFK, with a rating of 10.0.



In [91]:
str(response)

'The route with the highest overall rating is London to New York JFK, with a rating of 10.0.'

In [92]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("Wich Route	 has the least OverallRating ? ")

Added user message to memory: Wich Route	 has the least OverallRating ? 
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT Route, MIN(OverallRating) AS MinRating FROM reviews"
}
Got output: The route with the lowest overall rating is Heathrow to Marseille, with a rating of 1.0.



In [95]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("group OverallRating given by TypeOfTraveller	? ")

Added user message to memory: group OverallRating given by TypeOfTraveller	? 
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT TypeOfTraveller, AVG(OverallRating) AS AverageRating FROM reviews GROUP BY TypeOfTraveller"
}
Got output: The average ratings for different types of travelers are as follows:
- Business travelers have an average rating of 3.51.
- Couple leisure travelers have an average rating of 4.40.
- Family leisure travelers have an average rating of 4.81.
- Solo leisure travelers have an average rating of 5.29.



In [96]:
str(response)

'The average ratings for different types of travelers are as follows:\n- Business travelers have an average rating of 3.51.\n- Couple leisure travelers have an average rating of 4.40.\n- Family leisure travelers have an average rating of 4.81.\n- Solo leisure travelers have an average rating of 5.29.'

In [98]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                     name="sql_query",
                                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("Which Route has the lowest ReviewHeader	? ")

Added user message to memory: Which Route has the lowest ReviewHeader	? 
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT Route, MIN(ReviewHeader) AS LowestReviewHeader FROM reviews GROUP BY Route"
}
Got output: The lowest review header for each route is as follows:
- AMS to HKG via LHR: "most exhausting trip"
- ATH to KUL via LHR: "lounge at T5 is large and busy"
- ATL to GVA via LHR: "was a nightmare experience"
- Aberdeen to London Heathrow: "Fantastic flight down to Heathrow"
- Abu Dhabi to London: "operated by Air Belgium"
- Amman to London: "5 hours and 30 mins but no meal"
- Amsterdam to London: "delays and cancellations"
- Athens to London: "some sort of mean spirited joke"
- Athens to Philadelphia via London: "once again disappointed"
- Atlanta to London: "A very positive experience"
- BCN to SFO via LHR: "service was fine but underwhelming"
- BKK to CDG via LHR: "not fair for the fare"
- BKK to LHR: "really slipping down"
- BLR to LHR: "ser

In [100]:
response = agent.chat("Which Route has good reviews mentione in ReviewHeader	? ")

Added user message to memory: Which Route has good reviews mentione in ReviewHeader	? 
=== Calling Function ===
Calling function: sql_query with args: {
  "input": "SELECT Route, ReviewHeader FROM reviews WHERE ReviewHeader LIKE '%good%'"
}
Got output: Some of the reviews that mention "good" are: 
- Mumbai to London: "good, but not spectacular"
- London to New York: "Service was good"
- London Heathrow to Cape Town: "a good drinks and food service"
- San Francisco to London: "A good flight overall"
- London to Los Angeles: "Overall a good flight"
- Glasgow to Washington via London: "The service was very good"
- Mumbai to Boston via London: "the service was good"
- London to Johannesburg: "Overall a good flight"
- London to Rio De Janeiro: "another very good flight"
- London to Boston: "Food was fairly good both ways"
- London Heathrow to Abu Dhabi: "they were really good"
- London to New York JFK: "this was a good flight"
- London to Johannesburg: "Crew on this sector were good"
- Heat